In [2]:
#Importing

from IPython.display import display, clear_output, HTML, Image
import time
import random
random.seed(1)
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

#Defining functions

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
        post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
    
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return


event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()
    
    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1
    
    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)
            
            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
            
                    # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info


def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return

def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event(timeout=10)
    text_input.disabled = True
    return event['description']

In [3]:
#Defining the first set of ANS

def run_ANS_Set1():

    global set1_score, set1_time_total

    set1_score = 0
    set1_time_total = 0 
    
    mixfiles1 = ["b10_09.png", "y09_10.png", "b14_10.png", "y10_14.png","b14_12.png", "y12_14.png","b14_11.png", "y11_14.png","b16_12.png", "y12_16.png","b18_13.png", "y13_18.png","b18_14.png", "y14_18.png","b18_16.png", "y16_18.png","b20_15.png", "y15_20.png","b20_18.png", "y18_20.png","b21_18.png", "y18_21.png", "b12_09.png", "y09_12.png","b11_09.png","y09_11.png","b17_15.png","y15_17.png","b18_15.png","y15_18.png","b13_12.png","y12_13.png","b15_13.png","y13_15.png","b15_14.png","y14_15.png"]
    random.shuffle(mixfiles1)
    
    top_area = widgets.Output(layout={"height":"60px"})
    main_area = widgets.Output(layout={"height":"250px"})
    bottom_area = widgets.Output(layout={"height":"60px"})

    global test1_df
    
    test1_df = {
        'filename': [],
        'nL': [],
        'nR': [],
        'corr_ans': [],
        'ans': [],
        'score': [],
        'time_taken': []
    }
    
    btn1 = widgets.Button(description="B")
    btn2 = widgets.Button(description="Y")
    
    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)
    
    panel = widgets.HBox([btn1, btn2])
    top_area.append_display_data( HTML("<h1>ANS Test</h1>") )
    
    bottom_area.append_display_data(panel)
    
    display(top_area)
    display(main_area)
    display(bottom_area)
    
    n = len(mixfiles1)
    score = 0
    with main_area: print("The test will begin now")
    with main_area: time.sleep(1.5)
    with main_area: clear_output(wait = False)
    for i in range(n):
        picture = Image(mixfiles1[i], width = 400)
        with main_area: display(picture)
        with main_area: time.sleep(2.5)
        with main_area: clear_output(wait = False)
        time1_start = time.time()
        with main_area: print("Which colour had more dots?")
        result = wait_for_event(timeout=3)
        ans = 0
        if result['description'] == '':
            with main_area: display("Too slow!")
            ans = np.NaN
        elif result['description'].lower() ==  mixfiles1[i][0]:
            with main_area: display("Correct!")
            ans = 1
            set1_score = set1_score + 1
        else:
            with main_area: display("Wrong!")
                
        time1_end = time.time()
        time1_taken = time1_end - time1_start
        with main_area: print(f"\nTime taken: {time1_taken:.3f} seconds")
        set1_time_total += time1_taken
        
        test1_df['filename'].append(mixfiles1[i])
        test1_df['nL'].append(mixfiles1[i][1:3])
        test1_df['nR'].append(mixfiles1[i][4:6])
        test1_df['corr_ans'].append(mixfiles1[i][0])
        test1_df['ans'].append(result['description'].lower())
        test1_df['score'].append(ans)
        test1_df['time_taken'].append(time1_taken)
        wait_for_event(timeout=1.5)
        with main_area: clear_output()
            
    with bottom_area: clear_output()

    with main_area: print("Set 1 Statistics:")
    
    with main_area: print(f"\nYou scored {set1_score}")
        
    with main_area: print(f"\nTotal time taken: {set1_time_total:.3f} seconds")

    pct_set1 = (set1_score/36) * 100
    with main_area: print(f"\nPercentage of correct answers: {pct_set1:.3f}%")

    Avg_time = (set1_time_total)/36
    with main_area: print(f"\nAverage time taken per question: {Avg_time:.3f} seconds")

    time.sleep(10)

    return test1_df

In [4]:
#Defining the second set of ANS

def run_ANS_Set2():

    global set2_score, set2_time_total

    set2_score = 0
    set2_time_total = 0 

    mixfiles2 = ["b10_09.png", "y09_10.png", "b14_10.png", "y10_14.png","b14_12.png", "y12_14.png","b14_11.png", "y11_14.png","b16_12.png", "y12_16.png","b18_13.png", "y13_18.png","b18_14.png", "y14_18.png","b18_16.png", "y16_18.png","b20_15.png", "y15_20.png","b20_18.png", "y18_20.png","b21_18.png", "y18_21.png", "b12_09.png", "y09_12.png","b11_09.png","y09_11.png","b17_15.png","y15_17.png","b18_15.png","y15_18.png","b13_12.png","y12_13.png","b15_13.png","y13_15.png","b15_14.png","y14_15.png"]
    random.shuffle(mixfiles2)
    
    top_area = widgets.Output(layout={"height":"60px"})
    main_area = widgets.Output(layout={"height":"250px"})
    bottom_area = widgets.Output(layout={"height":"60px"})

    global test2_df
    
    test2_df = {
        'filename': [],
        'nL': [],
        'nR': [],
        'corr_ans': [],
        'ans': [],
        'score': [],
        'time_taken' : []
    }
    
    btn1 = widgets.Button(description="B")
    btn2 = widgets.Button(description="Y")
    
    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)
    
    panel = widgets.HBox([btn1, btn2])
    top_area.append_display_data( HTML("<h1>ANS Test</h1>") )
    
    bottom_area.append_display_data(panel)
    
    display(top_area)
    display(main_area)
    display(bottom_area)
    
    n = len(mixfiles2)
    score = 0
    with main_area: clear_output(wait = False)
    with main_area: print("The test will begin now")
    with main_area: time.sleep(1.55)
    with main_area: clear_output(wait = False)
    for i in range(n):
        picture = Image(mixfiles2[i], width = 400)
        with main_area: display(picture)
        with main_area: time.sleep(1.25)
        with main_area: clear_output(wait = False)
        time2_start = time.time()
        with main_area: print("Which colour had more dots?")
        result = wait_for_event(timeout=3)
        ans = 0
        if result['description'] == '':
            with main_area: display("Too slow!")
            ans = np.NaN
        elif result['description'].lower() ==  mixfiles2[i][0]:
            with main_area: display("Correct!")
            ans = 1
            set2_score = set2_score + 1
        else:
            with main_area: display("Wrong!")
        

        time2_end = time.time()
        time2_taken = time2_end - time2_start
        with main_area: print(f"\nTime taken: {time2_taken:.3f} seconds")
        set2_time_total += time2_taken
        
        test2_df['filename'].append(mixfiles2[i])
        test2_df['nL'].append(mixfiles2[i][1:3])
        test2_df['nR'].append(mixfiles2[i][4:6])
        test2_df['corr_ans'].append(mixfiles2[i][0])
        test2_df['ans'].append(result['description'].lower())
        test2_df['score'].append(ans)
        test2_df['time_taken'].append(time2_taken)
        time.sleep(1)
        with main_area: clear_output()
        
    with bottom_area: clear_output()

    with main_area: print("Set 2 Statistics:")
        
    with main_area: print(f"\nYou scored {set2_score}")
        
    with main_area: print(f"\nTotal time taken: {set2_time_total:.3f} seconds")
        
    pct_set2 = (set2_score/36) * 100
    with main_area: print(f"\nPercentage of correct answers: {pct_set2:.3f}%")
        
    Avg_time = (set2_time_total)/36
    with main_area: print(f"\nAverage time taken per question: {Avg_time:.3f} seconds")

    time.sleep(10)
    return test2_df

In [5]:
#defining the overall ANS test with both sets

def run_ANS_combined():
    
    newdata = {
    'id':"A",
    'age': 0,
    'sex': " ",
    'set1_score': 0,
    'set2_score': 0,
    'set1_time_total': 0,
    'set2_time_total': 0,
    'set1_results_json': ' ',
    'set2_results_json': ' '
    }
    
    data_consent_info = """DATA CONSENT INFORMATION:
     
    Please read:
    
    We wish to record your response data to an anonymised public data repository.
    
    Your data will be used for educational teaching purposes practising data analysis and visualisation.

    """
    
    print(data_consent_info)

    time.sleep(10)

    print("Please type yes in the box below if you consent to the upload.")
    
    result = text_input("> ")

    time.sleep(5)
    
    if result.lower() == "yes":
    
        print("\nThanks for your participation.")
    
        print("Please contact philip.lewis@ucl.ac.uk")
    
        print("If you have any questions or concerns")
    
        print("regarding the stored results.")
        
        time.sleep(5)
    
    else:
    
    # end code execution by raising an exception
    
        raise(Exception("User did not consent to continue test."))

    clear_output()
    
    id_instructions = """
    
    Create your anonymised ID:
    
    Please enter a random 4-letter User ID in the box below.
    
    This ID will be used to store your test results.
    
    """
    
    print(id_instructions)

    user_id = text_input("> ")

    time.sleep(20)

    clear_output(wait = True)
    
    print("User entered id:", user_id)

    sex = text_input("Please enter your sex (m - male, f - female):")
    
    age = text_input("What is your age? Please enter an integer:")

    clear_output(wait = True)
    
    ANS_instructions = """

    General Instructions: 
    
    This is the Approximate Number Sense (Test). This test will last for approximately 6-7 minutes.
    
    The test will involve two circles, placed side by side, containing coloured dots.

    Once the circles disappear, please pick the colour which had a greater number of dots. 
    """
    
    print(ANS_instructions)
    time.sleep(10)
    clear_output()
    
    set1_instructions = """

    Set 1 Instructions:
    
    The test will have two sets. In the first set, each image will be displayed for 2.5 seconds.
    
    For each set, you have a maximum of 3 seconds to answer the question once the image disappears.

    WARNING: Please do not double-click the buttons when completing the test!
    """

    print(set1_instructions)
    time.sleep(10)
    
    clear_output()
    
    run_ANS_Set1()

    clear_output()

    set2_instructions = """

    Set 2 Instructions:
    
    Set 2 will begin now. In this set, each image will be displayed for 1.25 seconds
    
    You have a maximum of 3 seconds to answer the question once the image disappears.

    WARNING: Please do not double-click the buttons when completing the test!
    """
    print(set2_instructions)
    time.sleep(10)
    clear_output()

    run_ANS_Set2()
    clear_output()
    print("Thank you for participating in the test!")

    Set1_Question_info = pd.DataFrame(test1_df)
    Set2_Question_info = pd.DataFrame(test2_df)

    newdata = {
            'id': [user_id],
            'age': [age],
            'sex': [sex],
            'set1_score': [set1_score],
            'set2_score': [set2_score],
            'set1_time_total': [set1_time_total],
            'set2_time_total': [set2_time_total],
            'set1_results_json': Set1_Question_info.to_json(),
            'set2_results_json': Set2_Question_info.to_json()        
            }
    
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSe5A2E0XojVHl2GEunHO2Dia91HkXRvr90CHe6DgLFa7iBN8Q/viewform?usp=sharing"
    send_to_google_form(newdata, form_url)
    
    return

In [ ]:
run_ANS_combined()

DATA CONSENT INFORMATION:
     
    Please read:
    
    We wish to record your response data to an anonymised public data repository.
    
    Your data will be used for educational teaching purposes practising data analysis and visualisation.

    
